### Principles of Superconducting Quantum Computers
Daniel D. Stancil, Gregory T. Byrd, Wiley, 2022  ISBN: [978-1119750727](https://bcs.wiley.com/he-bcs/Books?action=index&bcsId=12203&itemId=1119750725)

----

## Chapter 2: Physics of Single Qubit Gates
### Gates and Rotations

As described in section 2.2, the operation of a gate can be described as a rotation along the surface of the Bloch sphere. Clockwise rotations around the three axes are described by the following matrices:

$$R_x(\theta) = e^{-i \frac{\theta}{2} \sigma^x} = \begin{bmatrix} \cos \frac{\theta}{2} & -i \sin \frac{\theta}{2} \\ -i \sin \frac{\theta}{2} & \cos \frac{\theta}{2} \end{bmatrix}$$

$$R_y(\theta) = e^{-i \frac{\theta}{2} \sigma^y} = \begin{bmatrix} \cos \frac{\theta}{2} & - \sin \frac{\theta}{2} \\ \sin \frac{\theta}{2} & \cos \frac{\theta}{2} \end{bmatrix}$$

$$R_z(\theta) = e^{-i \frac{\theta}{2} \sigma^z} = \begin{bmatrix} e^{-i \theta/2} & 0 \\ 0 & e^{i \theta/2} \end{bmatrix}$$

Starting from the $|0\rangle$ position, we can get to any other point on the sphere with two rotations:
1. Rotate around the Y axis by an angle $\theta$.
1. Rotate around the Z axis by an angle $\phi$.

As derived in the book:

$$R_z(\phi)R_y(\theta)|0\rangle = e^{-i\phi/2}(\cos \frac{\theta}{2} |0\rangle + e^{i \phi} \sin \frac{\theta}{2}|1\rangle)$$

The *global phase* of $- \phi/2$ does not matter; in fact, two states that differ only by a global phase are indistinguishable. It's common, therefore, to consider angles $\theta$ and $\phi$ when viewing the Bloch sphere. (A global phase is analogous to rotating the entire sphere, rather than rotating around an axis.)


In [1]:
from ipywidgets import interact
import ipywidgets as widgets
from kaleidoscope import bloch_sphere
import numpy as np
from rotations import *  # functions specific to this notebook

In [2]:
# if the Bloch spheres stop rendering, try executing this cell
import plotly.io as pio
pio.renderers.default='notebook'

Use the text boxes below to convince yourself by entering angles for $\theta$ and $\phi$. You should be able to reach all other points of the sphere. Each angle is entered as a multiple of $\pi$ -- e.g., a value of $1.5$ for $\frac{3\pi}{2}$. Note that we use the `rz` and `ry` gates from Qiskit to perform the Z and Y rotations.

In [3]:
angley = widgets.BoundedFloatText(value = 0, min = -2, max = 2, step = 0.05, description='Y angle: \u03B8 / \u03C0')
anglez = widgets.BoundedFloatText(value = 0, min = -2, max = 2, step = 0.05, description='Z angle: \u03C6 / \u03C0')
sphere = widgets.interactive_output(show_yz, {'theta': angley, 'phi': anglez})
angles = widgets.VBox([angley, anglez])
angles.layout.align_items='center'
angles.layout.align_content='center'
widgets.HBox([sphere, angles])

While this works well when starting at $|0\rangle$, it's not a general transformation. Consider starting at position $|{+y}\rangle$: the Y rotation has no effect, so you could only reach points along the equator.

NOTE: In the text, the points at the extreme of the $y$-axis are known as $|i\rangle$ and $|{-i}\rangle$. Another common notation is $|r\rangle$ and $|l\rangle$ (for "right" and "left"). This is the notation used in the illustration below, and is used by Qiskit to initialize a state vector.

For a general gate description, we need three rotations, similar to the three axes of rotation of a classical object: pitch, roll, and yaw. There are many conventions for specifying a general rotation. We will use ZYZ, because $z$-axis rotations can be performed easily in superconducting quantum computers.

In the following, choose a starting point, and then specify three angles for the following rotations:
1. Rotate around the Z axis by angle $\lambda$.
1. Rotate around the Y axis by angle $\theta$.
1. Rotate around the Z axis by angle $\phi$.

Following the Qiskit convention, we call this `u3`, and it represents the most general single-qubit unitary gate, ignoring the global phase that results from performing the three rotations.

$$u3(\theta,\phi,\lambda) = \begin{bmatrix} \cos \frac{\theta}{2} & -e^{i\lambda} \sin \frac{\theta}{2} \\
                                e^{i\phi} \sin \frac{\theta}{2} & e^{i(\phi + \lambda)} \cos \frac{\theta}{2} 
                                \end{bmatrix}$$

In [4]:
style = {'description_width': '110px'}
pos = widgets.Dropdown(options=['0', '1', '+', '-', 'r', 'l'], value='+', description='start', style=style)
z1 = widgets.BoundedFloatText(value = 0, min = -2, max = 2, step = 0.05, description='1st Z angle: \u03BB / \u03C0',
                             style=style)
y = widgets.BoundedFloatText(value = 0, min = -2, max = 2, step = 0.05, description='Y angle: \u03B8 / \u03C0',
                            style=style)
z2 = widgets.BoundedFloatText(value = 0, min = -2, max = 2, step = 0.05, description='2nd Z angle: \u03C6 / \u03C0',
                             style=style)
sphere = widgets.interactive_output(show_zyz, {'start': pos, 'lbda': z1, 'theta': y, 'phi': z2})
params = widgets.VBox([pos, z1, y, z2])
params.layout.align_items='center'
params.layout.align_content='center'
widgets.HBox([sphere, params])

#### Exercises
Using the interactive illustration above, demonstrate the following equivalences described in the book for the Qiskit universal gate: $u3(\theta, \psi, \lambda).  Pay attention to the order of angles in the parameter list. Try with different starting points to convince yourself that the gate acts correctly.

$$ X = u3(\pi, 0, \pi)$$

$$ Y = u3(\pi, \pi/2, \pi/2)$$

$$ H = u3(\pi/2, 0, \pi)$$

----

In [5]:
import qiskit.tools.jupyter
%qiskit_version_table